In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.4MB/s 
     |████████████████████████████████| 3.3MB 57.9MB/s 
     |████████████████████████████████| 901kB 49.5MB/s 


In [2]:
# python imports
import numpy as np
import pandas as pd 
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import tqdm.notebook as tq

# huggingface
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

# inits
pd.set_option('display.max_colwidth', None)

In [22]:
# file import
file_name = "preprocessed_tweets_0.xlsx"
# tweets = pd.read_csv(f'drive/MyDrive/{file_name}')
# tweets = pd.read_pickle(f'drive/MyDrive/{file_name}')
tweets = pd.read_excel(f'drive/MyDrive/{file_name}')

In [23]:
# check if the file is loaded correctly
tweets.head()

,id,text
0,129855,⁦ ⁦ ⁦ primarycarehome event at faculty day. what a beautiful thank you wall. a thank why…
1,58704,- 2020 alternative lending state of the market report sponsored by will be out soon! sign up h…
2,17310,"⁦ australia: as industry voices call for a more sustainable approach, what are the future opportunities and challenges from this enforced slowdown for australia ? from retail to re-tale ⁦ ⁦"
3,457988,"' e-news just arrived in your inbox filled with support for business through challenging time. support hub, payi…"
4,33907,"- founder discusses digital finance forum, covid impact on fintech"


In [24]:
tweets.shape

(99932, 2)

In [7]:
# model init
task = 'sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [9]:
# simple example to see if the model works correctly
text = "in case you don’t speak bankese, we’ve translated it for you.because knowing how much it really costs to send money abr…"

encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) neutral 0.5745
2) negative 0.3552
3) positive 0.0703


In [10]:
ranking
# 2 - positive class
# 1 - neutral class
# 0 - negative class

array([1, 0, 2])

In [ ]:
# this can probably be done in more vectorised way
# df_scores = pd.DataFrame(columns=labels)

# for i in tq.tqdm(range(len(tweets['text'].iloc[:3]))):
# # for i in tq.tqdm(range(len(tweets['text']))):
#   try:
#     encoded_input = tokenizer(tweets['text'][i], return_tensors='pt')
#     output = model(**encoded_input)
#     scores = output[0][0].detach().numpy()
#     scores = softmax(scores)
#     values_to_add = {k: v for k, v in zip(labels, scores)}
#   except:
#     print(tweets['text'][i])
#     values_to_add = {k: np.nan for k in labels}

#   row_to_add = pd.Series(values_to_add)
#   df_scores = df_scores.append(row_to_add, ignore_index=True)

# it would take ~5h to complete
# time for plan B

In [11]:
!pip install datasets

     |████████████████████████████████| 204kB 8.6MB/s 
     |████████████████████████████████| 112kB 16.4MB/s 
     |████████████████████████████████| 245kB 12.3MB/s 


In [25]:
from datasets import Dataset
import torch

tweets_c = tweets.copy()
dataset = Dataset.from_pandas(tweets_c)

In [26]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [27]:
device

device(type='cuda')

In [29]:
encoded_dataset = dataset.map(lambda examples: tokenizer(examples['text'], padding='max_length', truncation=True, max_length=42), batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'], device=device)
dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=32)

In [30]:
all_scores = np.empty((0,3), int)
model = model.to(device)

for batch in dataloader:
  output = model(**batch)
  scores = output[0].cpu().detach().numpy()
  all_scores = np.vstack((all_scores, scores))

# done in 5 min

In [31]:
all_scores_softmax = softmax(all_scores, axis = 1)

In [33]:
all_scores_softmax.shape

(99932, 3)

In [52]:
id_from_encoded = encoded_dataset['id'].copy()

In [54]:
df_scores = pd.DataFrame(all_scores_softmax, columns = labels)
df_scores['id'] = id_from_encoded

In [55]:
# save to local disc
file_to_save = 'tweet_scores_0.csv'
df_scores.to_csv((f'drive/MyDrive/{file_to_save}'), index=False)